In [1]:
import sys
sys.path.append('../..')
import cortex_etl as c_etl

ma = c_etl.analysis_initial_processing("../../configs/2-PfrTransfer/2-PfrTransfer-6-3rdConnectionRemaining.yaml", loglevel="ERROR")
a_hex0 = ma.hex0_spikes
a_hexO1 = ma.hex_O1_spikes
c_etl.post_analysis(a_hex0)


----- Custom post analysis -----
----- Custom post analysis complete: 12.98s -----


# Fig 4G1

In [ ]:
a_hex0.analysis_config.custom['plot_rasters'] = True
c_etl.plot_rasters(a_hex0, custom_file_path='figures/Fig7A-Sim1', simulation_filter={"ca":1.05, "depol_stdev_mean_ratio": 0.4, "desired_connected_proportion_of_invivo_frs": 0.9})
c_etl.plot_rasters(a_hex0, custom_file_path='figures/Fig7A-Sim2', simulation_filter={"ca":1.1, "depol_stdev_mean_ratio": 0.2, "desired_connected_proportion_of_invivo_frs": 0.9})

# Video

In [2]:
c_etl.plot_rasters(a_hex0)


----- Plot rasters, window: conn_spont -----

----- Create raster videos, window: conn_spont -----


# Fig 4G2

In [ ]:
hm_dims = (a_hex0.analysis_config.custom['heatmap_dims']['hor_key'], 
           a_hex0.analysis_config.custom['heatmap_dims']['ver_key'], 
           a_hex0.analysis_config.custom['heatmap_dims']['x_key'], 
           a_hex0.analysis_config.custom['heatmap_dims']['y_key'])

c_etl.heatmap(a_hex0.custom['custom_simulations_post_analysis'], 
              "ei_corr_rval", 
              "figures/Fig7B-ei_corr_rval.pdf", 
              *hm_dims, 
              mask_key="bursting", 
              figsize=(8, 6))

# Fig 4I Left

In [ ]:
c_etl.plot_sim_stat_lines_all_sims(a_hex0, 
                                   a_hex0.custom['layer_wise_features'], 
                                   "ei_corr_rval", 
                                   {"bursting": False}, 
                                   "figures/Fig4Ileft-LayerWiseEI_NonBursting", 
                                   [c_etl.silico_layer_strings[:0:-1]], 
                                   ['Greys'], 
                                   show_in_vivo_FRs=False, 
                                   custom_x_lims=[-0.2, 1.0], 
                                   show_colorbar=True,
                                  file_types=[".pdf"],
                                  figsize=(1.6,2.25))

# Fig 4H

In [ ]:
# c_etl.flatspace_videos(a_hexO1)

a = a_hexO1


import os
from blueetl.parallel import call_by_simulation
from functools import partial

for flatspace_video_key in a.analysis_config.custom['flatspace_videos']:
    flatspace_video_opt = a.analysis_config.custom['flatspace_videos'][flatspace_video_key]
    flatspace_video_opt['vid_str'] = flatspace_video_opt['window'] + "_" + str(flatspace_video_opt['t_step']) + "_" + str(flatspace_video_opt['n_spatial_bins']) + "_" + str(flatspace_video_opt['temporal_smoothing_sigma'])
    flatspace_video_opt['video_output_root'] = str(a.figpaths.flatspace_videos) + "/" + flatspace_video_opt['vid_str'] + "/"
    os.makedirs(flatspace_video_opt['video_output_root'], exist_ok=True)

    dataframes={
        "circuits": a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']],
        "spikes": a.repo.report.df.etl.q(neuron_class="ALL", window=flatspace_video_opt['window']),
        "windows": a.repo.windows.df.etl.q(window=flatspace_video_opt['window']), 
        "neurons": a.repo.neurons.df.etl.q(neuron_class="ALL")}

    gids = a.repo.neurons.df.etl.q(circuit_id=0)['gid']
    locations = a.repo.simulations.df.loc[:, ['circuit', 'circuit_id', 'simulation_id']].iloc[0]['circuit'].cells.get(gids, ["x", "y", "z"])
    flat_locations = flatten_locations(locations, a.analysis_config.custom["flatmap"])

    results = call_by_simulation(a.repo.simulations.df.etl.q(ca=1.1, depol_stdev_mean_ratio=0.2, desired_connected_proportion_of_invivo_frs=0.9), 
                                    dataframes, 
                                    func=partial(single_flatspace_video, 
                                                flat_locations=flat_locations, 
                                                flatspace_video_opt=flatspace_video_opt, 
                                                analysis_config=a.analysis_config.custom,
                                                flatspace_path_pre='figures/Fig7D-correlated_flatspace.mp3', 
                                                images_dir='figures/Fig7D-correlated_flatspace/'),
                                    how='series')

# Fig 4G3

In [ ]:
import pandas as pd

hor_key="ca"; ver_key="depol_stdev_mean_ratio"; x_key="freq"; y_key="desired_connected_proportion_of_invivo_frs"
hm_dims = (hor_key, ver_key, x_key, y_key)

all_fft_dfs = pd.merge(a_hex0.custom['fft'].reset_index(), a_hex0.custom['custom_simulations_post_analysis'].reset_index()[['simulation_id', 'bursting']])
c_etl.heatmap(all_fft_dfs, "power",  "figures/Fig7E-FFT", *hm_dims, mask_key="bursting", figsize=(3.7, 2.1), show_colorbar=True, override_cmap='Greys')

In [ ]:
a_hex0.custom['custom_features_by_neuron_class'].columns
min_depol_mean_by_sim = a_hex0.custom['custom_features_by_neuron_class'].groupby(['simulation_id']).min()['depol_mean']
max_depol_mean_by_sim = a_hex0.custom['custom_features_by_neuron_class'].groupby(['simulation_id']).max()['depol_mean']
import numpy as np
a = np.mean(max_depol_mean_by_sim - min_depol_mean_by_sim)

min_depol_mean_by_neuron_class = a_hex0.custom['custom_features_by_neuron_class'].groupby(['neuron_class']).min()['depol_mean']
max_depol_mean_by_neuron_class = a_hex0.custom['custom_features_by_neuron_class'].groupby(['neuron_class']).max()['depol_mean']
b = np.mean(max_depol_mean_by_neuron_class - min_depol_mean_by_neuron_class)

a/b


# Fig 6F

In [ ]:
import sys
sys.path.append('../..')
import cortex_etl as c_etl

ma2 = c_etl.analysis_initial_processing("../../configs/8-FRDistributionTest/8-FRDistributionTest-0-Test.yaml", loglevel="ERROR")
a2_hex0 = ma2.hex0_spikes
c_etl.post_analysis(a2_hex0)
a = a2_hex0

In [ ]:
# neuron_classes = ['L1_INH', 'L23_EXC', 'L23_INH', 'L23_PV', 'L23_SST', 'L23_Htr3a', 'L4_EXC', 'L4_INH', 'L4_PV', 'L4_SST', 'L4_Htr3a', 'L5_EXC', 'L5_INH', 'L5_PV', 'L5_SST', 'L5_Htr3a', 'L6_EXC', 'L6_INH', 'L6_PV', 'L6_SST', 'L6_Htr3a']
neuron_classes = ['L1_INH', 'L23_EXC', 'L23_INH', 'L4_EXC', 'L4_INH', 'L5_EXC', 'L5_INH', 'L6_EXC', 'L6_INH']
sim = a_hex0.repo.simulations.df.etl.q(ca=1.05, desired_connected_proportion_of_invivo_frs=0.9, depol_stdev_mean_ratio=0.4).iloc[0]
filtered_df = a_hex0.features.by_gid.df.reset_index().etl.q(neuron_class=neuron_classes, simulation_id=sim['simulation_id'])
filtered_df.loc[:, 'layer'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[0], axis = 1)
filtered_df.loc[:, 'neuron_type'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[1], axis = 1)
spiking_filtered_df = filtered_df.drop(filtered_df[filtered_df.mean_firing_rates_per_second == 0.0].index)

import matplotlib.pyplot as plt
import seaborn as sns
bw = .1
# hue_order= ["EXC", "INH", "PV", "SST", "Htr3a"]
# palette = {"EXC":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_EXC"]['color'], 
#            "INH":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_INH"]['color'], 
#            "PV":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_PV"]['color'], 
#            "SST": c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_SST"]['color'], 
#            "Htr3a":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_Htr3a"]['color']}
hue_order= ["EXC", "INH"]
palette = {"EXC":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_EXC"]['color'], 
           "INH":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_INH"]['color']}
ylabel = 'MFR (spikes/s)'

plt.figure(figsize=(2, 2.5))
# sns.violinplot(data=filtered_df, x="layer", y="mean_firing_rates_per_second", hue="neuron_type", hue_order=hue_order, palette=palette, scale='width', bw=bw, linewidth=0.01)
# sns.boxplot(data=filtered_df, x="layer", y="mean_firing_rates_per_second", hue="neuron_type", hue_order=hue_order, palette=palette)
sns.boxplot(data=filtered_df, y="layer", x="mean_firing_rates_per_second", width=0.25, hue="neuron_type", hue_order=hue_order, palette=palette, fliersize=0)
# plt.gca().set_ylim([0.0, 8.0])
plt.gca().set_xlim([0.0, 3.0])
plt.gca().set_ylabel(ylabel)
plt.savefig("figures/Fig7F-MFR-Violin.pdf")
plt.show()

plt.figure(figsize=(2, 2.5))
# sns.violinplot(data=spiking_filtered_df, x="layer", y="mean_firing_rates_per_second", hue="neuron_type", hue_order=hue_order, palette=palette, scale='width', bw=bw, linewidth=0.01)
sns.boxplot(data=spiking_filtered_df, y="layer", x="mean_firing_rates_per_second", width=0.25, hue="neuron_type", hue_order=hue_order, palette=palette)
# plt.gca().set_ylim([0.0, 10.0])
plt.gca().set_xlim([0.0, 3.0])
plt.gca().set_xlabel(ylabel)
plt.savefig("figures/Fig7F-MFR-SpikingOnly-Violin.pdf")
plt.show()


In [ ]:
# neuron_classes = ['L1_INH', 'L23_EXC', 'L23_INH', 'L23_PV', 'L23_SST', 'L23_Htr3a', 'L4_EXC', 'L4_INH', 'L4_PV', 'L4_SST', 'L4_Htr3a', 'L5_EXC', 'L5_INH', 'L5_PV', 'L5_SST', 'L5_Htr3a', 'L6_EXC', 'L6_INH', 'L6_PV', 'L6_SST', 'L6_Htr3a']
neuron_classes = ['ALL_INH'] #, 'ALL_EXC'
# print(a2_hex0.repo.simulations.df)
sim = a2_hex0.repo.simulations.df.etl.q(ca=1.05, desired_connected_proportion_of_invivo_frs=0.3, depol_stdev_mean_ratio=0.4).iloc[0]
filtered_df = a2_hex0.features.by_gid.df.reset_index().etl.q(neuron_class=neuron_classes, simulation_id=sim['simulation_id'])
filtered_df.loc[:, 'layer'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[0], axis = 1)
filtered_df.loc[:, 'neuron_type'] = filtered_df.apply(lambda row: row['neuron_class'].split('_')[1], axis = 1)
spiking_filtered_df = filtered_df.drop(filtered_df[filtered_df.mean_firing_rates_per_second == 0.0].index)

import matplotlib.pyplot as plt
import seaborn as sns
bw = .1
hue_order= ["EXC", "INH"]
palette = {"EXC":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_EXC"]['color'], 
           "INH":c_etl.NEURON_CLASS_LAYERS_AND_SYNAPSE_CLASSES["L5_INH"]['color']}
ylabel = 'MFR (spikes/s)'

# sns.histplot(filtered_df, x="neuron_class", hue="mean_firing_rates_per_second")

# sns.histplot(data=filtered_df, x="neuron_class", kde=True)

# sns.histplot(spiking_filtered_df, x="mean_firing_rates_per_second", hue="neuron_class", element="step", stat="density", common_norm=False)
# sns.histplot(filtered_df, x="mean_firing_rates_per_second", hue="species", element="poly")

sns.histplot(data=spiking_filtered_df, x="mean_firing_rates_per_second", hue='neuron_class', kde=True)
plt.gca().set_xlim([0.0, 5.0])